# **Quantum GAN**

In [1]:
pip install pennylane

  Using cached PennyLane-0.21.0-py3-none-any.whl (800 kB)
  Using cached autoray-0.2.5-py3-none-any.whl (16 kB)
  Using cached semantic_version-2.6.0-py3-none-any.whl (14 kB)
  Using cached cachetools-5.0.0-py3-none-any.whl (9.1 kB)
  Using cached autograd-1.3.tar.gz (38 kB)
  Using cached PennyLane_Lightning-0.21.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47988 sha256=ef4ddbfadee648c8bf381dc44025652c6773a703ed20226f084a38240a17911b
  Stored in directory: /home/jovyan/.cache/pip/wheels/85/f5/d2/3ef47d3a836b17620bf41647222825b065245862d12aa62885
Successfully built autograd
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pennylane as qganPenny
from pennylane import numpy as numcal
from pennylane.optimize import GradientDescentOptimizer as GDO

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
device = qganPenny.device('default.qubit', wires=3)

In [4]:
def GetQGANReal(phi, theta, omega):
    qganPenny.Rot(phi, theta, omega, wires=0)

In [5]:
def GetQGANGenerator(wireArray):
    qganPenny.RX(wireArray[0], wires=0)
    qganPenny.RX(wireArray[1], wires=1)
    qganPenny.RY(wireArray[2], wires=0)
    qganPenny.RY(wireArray[3], wires=1)
    qganPenny.RZ(wireArray[4], wires=0)
    qganPenny.RZ(wireArray[5], wires=1)
    qganPenny.CNOT(wires=[0,1])
    qganPenny.RX(wireArray[6], wires=0)
    qganPenny.RY(wireArray[7], wires=0)
    qganPenny.RZ(wireArray[8], wires=0)

In [6]:
def GetQGANDiscriminator(wireArray):
    qganPenny.RX(wireArray[0], wires=0)
    qganPenny.RX(wireArray[1], wires=2)
    qganPenny.RY(wireArray[2], wires=0)
    qganPenny.RY(wireArray[3], wires=2)
    qganPenny.RZ(wireArray[4], wires=0)
    qganPenny.RZ(wireArray[5], wires=2)
    qganPenny.CNOT(wires=[1,2])
    qganPenny.RX(wireArray[6], wires=2)
    qganPenny.RY(wireArray[7], wires=2)
    qganPenny.RZ(wireArray[8], wires=2)

In [19]:
@qganPenny.qnode(device)
def GetQGANRealDiscCircuit(phi, theta, omega, discWeights):
    GetQGANReal(phi, theta, omega)
    GetQGANDiscriminator(discWeights)
    return qganPenny.expval(qganPenny.PauliZ(2))

In [20]:
@qganPenny.qnode(device)
def GetQGANDiscCircuit(genWeights, discWeights):
    GetQGANGenerator(genWeights)
    GetQGANDiscriminator(discWeights)
    return qganPenny.expval(qganPenny.PauliZ(2))

In [9]:
def GetQGANRealTrue(discWeights):
    trueDiscriminatorOutput = GetQGANRealDiscCircuit(phi, theta, omega, discWeights)
    probabilityRealTrue = (trueDiscriminatorOutput + 1) / 2
    return probabilityRealTrue

In [10]:
def GetQGANFakeTrue(genWeights, discWeights):
    fakeDiscriminatorOutput = GetQGANDiscCircuit(genWeights, discWeights)
    probabilityFakeTrue = (fakeDiscriminatorOutput + 1) / 2
    return probabilityFakeTrue 

In [11]:
def GetQGANDiscriminatorCost(discWeights):
    cost = GetQGANFakeTrue(genWeights, discWeights) - GetQGANRealTrue(discWeights) 
    return cost

In [12]:
def GetQGANGeneratorCost(genWeights):
    return -GetQGANFakeTrue(genWeights, discWeights)

In [13]:
phi = numcal.pi / 6
theta = numcal.pi / 2
omega = numcal.pi / 7

In [14]:
numcal.random.seed(0)
epsValue = 1e-2
genWeights = numcal.array([numcal.pi] + [0] * 8) + numcal.random.normal(scale=epsValue, size=[9])
discWeights = numcal.random.normal(size=[9])

In [17]:
gdo = GDO(0.1)
print("Training the discriminator ")

Training the discriminator 


In [21]:
for iteration in range(50):
    discriminator_weights = gdo.step(GetQGANDiscriminatorCost, discWeights) 
    discriminator_cost = GetQGANDiscriminatorCost(discriminator_weights)
    if iteration % 5 == 0:
        print("Iteration num {}: discriminator cost is = {}".format(iteration+1, discriminator_cost))

Iteration num 1: discriminator cost is = -0.10942017805789106
Iteration num 6: discriminator cost is = -0.10942017805789106
Iteration num 11: discriminator cost is = -0.10942017805789106
Iteration num 16: discriminator cost is = -0.10942017805789106
Iteration num 21: discriminator cost is = -0.10942017805789106
Iteration num 26: discriminator cost is = -0.10942017805789106
Iteration num 31: discriminator cost is = -0.10942017805789106
Iteration num 36: discriminator cost is = -0.10942017805789106
Iteration num 41: discriminator cost is = -0.10942017805789106
Iteration num 46: discriminator cost is = -0.10942017805789106


In [23]:
print("discriminator - real true: ", GetQGANRealTrue(discWeights))
print("discriminator - fake true: ", GetQGANFakeTrue(genWeights, discWeights))
print("Training the generator.")

discriminator - real true:  0.18997763559951558
discriminator - fake true:  0.13399334520305395
Training the generator.


In [24]:
for iteration in range(200):
    generator_weights = gdo.step(GetQGANGeneratorCost, genWeights)
    generator_cost = -GetQGANGeneratorCost(generator_weights)
    if iteration % 5 == 0:
        print("Iteration num {}: generator cost is = {}".format(iteration, generator_cost))

Iteration num 0: generator cost is = 0.13399341778096302
Iteration num 5: generator cost is = 0.13399341778096302
Iteration num 10: generator cost is = 0.13399341778096302
Iteration num 15: generator cost is = 0.13399341778096302
Iteration num 20: generator cost is = 0.13399341778096302
Iteration num 25: generator cost is = 0.13399341778096302
Iteration num 30: generator cost is = 0.13399341778096302
Iteration num 35: generator cost is = 0.13399341778096302
Iteration num 40: generator cost is = 0.13399341778096302
Iteration num 45: generator cost is = 0.13399341778096302
Iteration num 50: generator cost is = 0.13399341778096302
Iteration num 55: generator cost is = 0.13399341778096302
Iteration num 60: generator cost is = 0.13399341778096302
Iteration num 65: generator cost is = 0.13399341778096302
Iteration num 70: generator cost is = 0.13399341778096302
Iteration num 75: generator cost is = 0.13399341778096302
Iteration num 80: generator cost is = 0.13399341778096302
Iteration num 85

In [25]:
print("discriminator - real true: ", GetQGANRealTrue(discWeights))
print("discriminator -  fake true: ", GetQGANFakeTrue(genWeights, discWeights))
print("Cost is: ", GetQGANDiscriminatorCost(discriminator_weights))

discriminator - real true:  0.18997763559951558
discriminator -  fake true:  0.13399334520305395
Cost is:  -0.10942017805789106
